<a href="https://colab.research.google.com/github/vividu/202406CSV/blob/main/GB886_IV_6_LogisticRegressionExampleConfusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression

In this tutorial, we introduce one of the most basic *binary classifiers*: **Logistic Regression**.

Assume we are given predictors $X_i$ and outcome variables $Y_i \in \{0,1\}$.  Remember that we are trying to assess
$$
p(x_0)= \mathbb{P}(Y=1|X=x_0).
$$
A sensible prediction for given data $x_0$ is then:
$$
\hat{Y} = \left\{\begin{array}{l} 0 \text{ if }p(x_0) \leq 0.5,\\ 1\text{ if }p(x_0) > 0.5. \end{array}\right.
$$

For instance, let's assume that $X$ is two-dimensional consisting of height and weight, and that we attempt to predict sex $Y$---say 1 for male and 0 for females.  Hence, $p(x)$ is the probability for male given $x =\{\text{height, weight}\}$, and the probability of being female is $1-p(x).$

This is the idea behind **Logistic regression** imposes a functional form on $p(x)$:
$$
p(x) = \frac{e^{x\beta}}{1+e^{x\beta}} = \frac{e^{\beta_0 + \beta_1 x_1 + ... + \beta_p x_p}}{1+e^{\beta_0 + \beta_1 x_1 + ... + \beta_p x_p}}
$$

For instance, in our male/female example, we would have:
$$
\text{Proability Male} = p(\underbrace{\text{height},\text{weight}}_x)=\frac{e^{\beta_0+\beta_1\times\text{height}+\beta_2\times\text{weight}}}{1+e^{\beta_0+\beta_1\times\text{height}+\beta_2\times\text{weight}}}.
$$

Our predictive modeling process then works as follows:
* We need to collect/find training data $(y_1,x_1),\,(y_2,x_2),\ldots,\,(y_N,x_N)$, in our example consisting of information on individual's sex and their height and weight.
* Based on the data, we use *Maximum Likelihood Estimation* to fit the model, i.e., to find an *estimate* fore the parameter vector $\beta=(\beta_0,\beta_1,\beta_2)'$.
* Then, we can predict the probability of being male/female for a given (height,weight) combination.


## Application

As always, let's start with importing the libraries:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm

### Data

Let's look at the dataset from a [study by Caroline Davis](https://www.sciencedirect.com/science/article/pii/019566639090096Q) that has data on the height and weight of 200 individuals.

Let's load the data:

In [ ]:
!git clone https://github.com/danielbauer1979/MSDIA_PredictiveModelingAndMachineLearning.git

In [ ]:
hw_data = pd.read_csv('MSDIA_PredictiveModelingAndMachineLearning/GB886_IV_3_Davis.csv')

And let's take a look:

In [ ]:
hw_data.head()

Let's recast the `sex` variable as a dummy variable, because that's the input for logistic regression packages.

In [ ]:
hw_data['sex'] = pd.get_dummies(hw_data['sex'],drop_first=True)
hw_data.head()

And let's visualize the data by plotting a scatterplot of height (in cm, on x-axis) and weight (in kg, on y-axis) and showing the labels by coloring the points:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
# plot x,y data with c as the color vector, set the line width of the markers to 0
ax.scatter(hw_data['height'], hw_data['weight'], c= hw_data['sex'])

Clearly, sex male is associated with larger height and weight.

### Fitting Logistic Regression Model

Let's run our classification model, where once again we use statsmodels for convenient output:

In [ ]:
X = hw_data[['height','weight']]
X = sm.add_constant(X)
y = hw_data[['sex']]

logit_mod = sm.Logit(y, X)
logit_mod_res = logit_mod.fit()

print(logit_mod_res.summary())

So, we see that the coefficients for height and weight are positive. That means taller people are more likely to be labeled as male. But how do we think about the actual interpretation?

To illustrate, consider two individuals that are 55kg and one is 170cm tall (person A), the other one is 175 cm tall (person B). Let's get the predicted probablity for them being sex male from our model:

In [ ]:
logit_mod_res.predict([1,170,55]) #person A

In [ ]:
logit_mod_res.predict([1,175,55]) #person B

As we indicated, we can transpate predicted probabilities into odds. Let's evaluate the "oddsn for being male" for person A and person B:

In [ ]:
oddsA = logit_mod_res.predict([1,170,55])/(1-logit_mod_res.predict([1,170,55]))
oddsA

In [ ]:
oddsB = logit_mod_res.predict([1,175,55])/(1-logit_mod_res.predict([1,175,55]))
oddsB

Now, we said that in logistic regression, the log-odds  are linear. So let's calculate the log-odds for person B minus the log-odds for person A divided by 5, because person B is 5 cm taller:

In [ ]:
(np.log(oddsB)-np.log(oddsA))/5

...and that's exactly our coefficient $\beta_1$. So, the interpretation is as follows: For every cm of height, the log-odds for $Y=1$ increase by $\beta_1$. Or, equivakently, for every cm of height, the odds increase by a factor of $e^{\beta_1}$:

In [ ]:
np.exp((np.log(oddsB)-np.log(oddsA))/5)

### Prediction

Let's predict the probability for being male in our sample:

In [ ]:
pres = logit_mod_res.get_prediction()
pres.summary_frame()

Let's compare out predicted probabiloties (x-axis) and the actual outcomes (y-axis):

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(pres.predicted,y)
plt.show()


We can also get the predictions for an observation outside of our sample, e.g., consider an individual that is 180cm tall and weighs 85kg:

In [ ]:
pres = logit_mod_res.get_prediction([1,180,85])

In [ ]:
pres.summary_frame()

So, due to parameter error, our confidence band for the predicted probability is between 98.1% and 99.96%. So, clarly we would label this observation as $\hat{y}=1$ (male). However, note that there is a chance we are wrong, and given parameter errors our chance of being wrong may be close to 1.9%.

### Evaluating our Classifier: Confusion Table

To evaluate our classifier, we can determine the confusion table for our predictions. Recall that the confusion table or matrix arranges the predicted class from left to right and the actual class on the vertical for a 2-by-2 table:

$$
\begin{array}{|l|c|c|}
\hline
 \text{Actual\Predicted} &  \text{False, } \hat{y}=0 & \text{True, } \hat{y}=1\\
 \hline
 \text{False, } y=0 & \text{TN} & \text{FP} \\
 \text{True, } y=1 & \text{FN} & \text{TN}\\
 \hline
\end{array}
$$

Often, one also includes the row and column sums.

Let's take a look:

In [ ]:
pred = (logit_mod_res.predict(X) > 0.5)
conf_mat = pd.crosstab(y['sex'], pred, rownames=['Actual Sex'], colnames=['Predicted Sex'])
# Add row and column sums
conf_mat.loc['Column_Total']= conf_mat.sum(numeric_only=True, axis=0)
conf_mat.loc[:,'Row_Total'] = conf_mat.sum(numeric_only=True, axis=1)
print(conf_mat)

As we described, we can determine various metrics based on the confusion table:

In [ ]:
TPR = 78 / 88 # True-Positive Rate
TNR = 104 / 112 # True-Negative Rate
FPR = 8 / 112 # False-Positive Rate = 1 - TNR
FNR = 10 / 88 # False-Negative Rate = 1 - TPR
MCR = (8+10)/200 # Miss Classification Rate
ACC = (78+104)/200 # Accuracy
print('TPR =', TPR)
print('TNR =', TNR)
print('FPR =', FPR)
print('FNR =', FNR)
print('MCR =', MCR)
print('ACC =', ACC)